<a href="https://colab.research.google.com/github/raafitt/GoogleColab/blob/neural-worker/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%BE%D1%82%D1%80%D1%83%D0%B4%D0%BD%D0%B8%D0%BA_%2B_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Нейросотрудники представляют собой рекрутеров Озон и Т-Банк. Обучаются на табличных данных

In [ ]:
!pip install openai gradio tiktoken langchain langchain-openai langchain-community chromadb faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.9 MB/s eta 0:00:0

In [ ]:
models = [
              {
                "doc": "https://docs.google.com/spreadsheets/d/1_R3hZN6M5STYqK2sCh9vtlC0m7L5s8g_/edit?gid=1950053208#gid=1950053208",
                "prompt": '''Ты менеджер по подбору персонала Т-Банк, к тебе могут обращаться соискатели с вопросами по поводу вакансий. Перед тобой документ, в котором есть колонки "Название вакансии","График работы","Формат работы",
                "Тип оформления","Бенефиты для сотрудников","Чем предстоит заниматься","Кого мы ищем"
                        Отвечай на вопросы на основе данных в документе, от себя ничего не выдумывай: ''',
                "name": "рекрутер Т-Банк",
                "query": "Какие вакансии есть?"
              },

               {
                "doc": "https://docs.google.com/spreadsheets/d/1UZHbBCdegN694atmKDgFEYFzos89JlXuylFfqHy0tJo/edit?usp=sharing",
                "prompt": '''Ты сотрудник по подбору персонала. Перед тобой документ, в котором есть колонки "Адрес", "Средний пробег на курьера в день","Ср.доход в день","Время оказания услуги",
                "Подработка","Смена","Вакансия Авто, количество", "Вакансия велосипед, количество" в которых содержится информация для курьера Вкус Вилл. Документ в формате csv, структурируй его.
                Если запрошенного пользователя нет в документе, кратко отвечай, что в данный момент информация по данному городу отсутствует
                        Твоя задача ответить на вопросы сосискателя используя информацию из документа.
                        Документ: ''',
                "name": "рекрутер Озон",
                "query": "Какой график работы в Москве?"
              },

            ]


In [ ]:
import getpass # для работы с паролями
import os      # для работы с окружением и файловой системой

# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")

Введите OpenAI API Key:··········


In [ ]:
# Блок библиотек фреймворка LangChain

# Работа с документами в langchain
from langchain.docstore.document import Document
# Эмбеддинги для OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
# Доступ к векторной базе данных
from langchain.vectorstores import Chroma
# Разделение текста на куски или чанки (chunk)
from langchain.text_splitter import CharacterTextSplitter
import chromadb
# Отправка запросов
import requests
import pandas as pd
import numpy as np

#Доступ к OpenAI
from openai import OpenAI

# Отприсовка интерфейса с помощью grad
import gradio as gr

# Библиотека подсчёта токенов
# Без запроcов к OpenAI, тем самым не тратим деньги на запросы
import tiktoken
import faiss
# Для работы с регулярными выражениями
import re

Для оптимизации улучшен поиск в векторном пространстве, для этого применен фреймворк FAISS. Facebook AI Research Similarity Search – разработка команды Facebook AI Research для быстрого поиска ближайших соседей и кластеризации в векторном пространстве. Метод create_embedding теперь создает и сохраняет вектора и добавляет их в FAISS.

В связи с особенностью предоставления данных таблицами, подаваемые на вход модели фрагменты одноуровневые, так как в таблицах информация может повторяться

In [ ]:
# Объявляем класс нейро-сотрудника
class GPT():
    def __init__(self, model="gpt-3.5-turbo"):
        self.log = ''
        self.model = model
        self.search_index = None
        self.embedded_docs = []  # Хранение документов для последующего извлечения по индексу
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    def load_search_indexes(self, url):
        # Преобразование URL Google Sheets в CSV
        pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'
        replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'
        new_url = re.sub(pattern, replacement, url)

        # Чтение данных и преобразование в строки
        rows_as_strings = ''
        df = pd.read_csv(new_url)
        for _, row in df.iterrows():
            row_str = ", ".join([f"{col}: {row[col]}" for col in df.columns])
            rows_as_strings += 'следующая вакансия ' + row_str

        # Создание векторного индекса с использованием FAISS
        return self.create_embedding(rows_as_strings)


    def num_tokens_from_string(self, string):
        encoding = encoding_for_model(self.model)
        return len(encoding.encode(string))

    def create_embedding(self, data):
        # Разбивка данных на части
        source_chunks = []
        splitter = CharacterTextSplitter(separator="\n", chunk_size=2048, chunk_overlap=0)
        for chunk in splitter.split_text(data):
            doc = Document(page_content=chunk, metadata={})
            source_chunks.append(doc)
            self.embedded_docs.append(doc)  # Добавляем документ в список для последующего доступа

        # Создание и добавление векторов в FAISS
        embeddings = [OpenAIEmbeddings().embed_query(doc.page_content) for doc in source_chunks]
        dimension = len(embeddings[0])
        self.search_index = faiss.IndexFlatL2(dimension) #Задание размерности входных векторов
        self.search_index.add(np.array(embeddings).astype('float32')) #добавление векторов в базу

        self.log += f'Количество токенов в документе: {self.num_tokens_from_string(" ".join([x.page_content for x in source_chunks]))}\n'
        self.log += 'Данные из документа загружены в векторную базу данных и FAISS индекс\n'
        return self.search_index

    def answer_index(self, system, topic, temp=1):
        if not self.search_index or not self.search_index:
            self.log += 'Модель необходимо обучить!\n'
            return ''

        # Поиск по векторной базе FAISS
        query_embedding = OpenAIEmbeddings().embed_query(topic)
        D, I = self.search_index.search(np.array([query_embedding]).astype('float32'), k=5) #Возвращает результат: дистанции и индексы

        # Получение документов по идентификаторам из базы
        docs = [self.embedded_docs[i] for i in I[0]]
        message_content = f"Описание вакансий. Найденная информация из базы: {[doc.page_content for doc in docs]}"

        self.log += f'message_content={message_content}\n'
        messages = [
            {"role": "system", "content": f"{system}\n{message_content}"},
            {"role": "user", "content": topic}
        ]

        # Запрос к языковой модели
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=temp
        )

        self.log += f'\nТокенов использовано всего (вопрос): {completion.usage.prompt_tokens}\n'
        self.log += f'Токенов использовано всего (вопрос-ответ): {completion.usage.total_tokens}\n'

        return completion.choices[0].message.content


In [ ]:
import re
import gradio as gr

# Объявляем экземпляр класса GPT и передаем ему в конструктор модель LLM
gpt = GPT("gpt-3.5-turbo")


# Gradio позволяет объединять элементы в блоки
blocks = gr.Blocks()

# Работаем с блоком
with blocks as demo:
    # Объявляем элемент выбор из списка
    subject = gr.Dropdown([(elem["name"], index) for index, elem in enumerate(models)], label="Данные")
    # Поля для отображения информации, связанной с выбранной моделью
    name = gr.Label(show_label=False)
    prompt = gr.Textbox(label="Промт", interactive=True)
    link = gr.HTML()
    query = gr.Textbox(label="Запрос к LLM", interactive=True)

    # Функция на выбор нейро-сотрудника в models
    def onchange(dropdown):
        try:
            # Проверка, что индекс выбора корректен
            if dropdown not in range(len(models)):
                raise ValueError("Некорректное значение выбора. Пожалуйста, выберите модель из списка.")

            # Получение данных из models
            selected_model = models[dropdown]
            return [
                selected_model['name'],
                re.sub(r'\t+|\s\s+', ' ', selected_model['prompt']),
                selected_model['query'],
                f"<a target='_blank' href='{selected_model['doc']}'>Документ для обучения</a>"
            ]
        except Exception as e:
            print(f"Ошибка при выборе модели: {e}")
            return ["Ошибка", "Ошибка", "Ошибка", ""]

    # При изменении значения в поле списка subject вызывается функция onchange
    subject.change(onchange, inputs=[subject], outputs=[name, prompt, query, link])

    # Строку в gradio можно разделить на столбцы (каждая кнопка в своем столбце)
    with gr.Row():
        train_btn = gr.Button("Обучить модель")
        request_btn = gr.Button("Запрос к модели")

    # Функция обучения
    def train(dropdown):
        try:
            # Проверка корректности индекса выбора модели
            if dropdown not in range(len(models)):
                raise ValueError("Некорректное значение выбора. Пожалуйста, выберите модель из списка.")

            # Загрузка документа и логгирование
            gpt.load_search_indexes(models[dropdown]['doc'])
            return gpt.log
        except Exception as e:
            print(f"Ошибка при обучении модели: {e}")
            return "Ошибка при обучении. Проверьте выбранные данные и повторите попытку."

    # Вызываем метод запроса к языковой модели из класса GPT
    def predict(p, q):
      try:
          # Проверка корректности полей prompt и query
          if not p.strip():
              raise ValueError("Поле промта не может быть пустым.")
          if not q.strip():
              raise ValueError("Поле запроса не может быть пустым.")

          # Вызов метода ответа от LLM
          result = gpt.answer_index(p, q)

          # Проверка успешности ответа
          if not result:
              raise RuntimeError("Ответ от модели пустой. Проверьте запрос и попробуйте снова.")

          # возвращает список из ответа от LLM и log от класса GPT
          return [result, gpt.log]

      except ValueError as ve:
          error_msg = f"Ошибка ввода: {ve}"
          print(error_msg)
          return [error_msg, ""]

      except RuntimeError as re:
          error_msg = f"Ошибка при получении ответа от LLM: {re}"
          print(error_msg)
          return [error_msg, gpt.log]

      except Exception as e:
          error_msg = f"Неизвестная ошибка при запросе к модели: {e}"
          print(error_msg)
          return [error_msg, gpt.log]

    # Выводим поля response с ответом от LLM и log (вывод сообщений работы класса GPT) на 2 колонки
    with gr.Row():
        response = gr.Textbox(label="Ответ LLM") # Текстовое поле с ответом от LLM
        log = gr.Textbox(label="Логирование")    # Текстовое поле с выводом сообщений от GPT


    # При нажатии на кнопку train_btn запускается функция обучения train_btn с параметром subject
    # Результат выполнения функции сохраняем в текстовое поле log - лог выполнения
    train_btn.click(train, [subject], log)

    # При нажатии на кнопку request_btn запускается функция отправки запроса к LLM request_btn с параметром prompt, query
    # Результат выполнения функции сохраняем в текстовые поля  response - ответ модели, log - лог выполнения
    request_btn.click(predict, [prompt, query], [response, log])

# Запуск приложения
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f80aed1e9cfe6f7a1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
